## Main: Data Cleaner
### Choose clean non-pooled data only

In [3]:
import pandas as pd
import json

clean_file_path = "clean_trips.json"
pooling_data = pd.read_csv("careems data/anon_pooling_jan_24_amman.csv")

filtered_pooling_data = pooling_data[pooling_data['day'] == '2024-01-31']


with open("fixed_traj_new.json", "r") as json_traj_file:
    new_data = [json.loads(line) for line in json_traj_file]  #read each line as a JSON object



with open(clean_file_path, 'w', encoding='utf-8') as clean_file :
    for entry in new_data:
        entry_time = entry["time"]
        entry_trip_id = entry["trip_id"]
        entry_driver_id = entry["driverID"]
                    
        for _, row in filtered_pooling_data.iterrows():
            pool_time = float(row["captain_engagement_time"] * 60)
            pool_trip_id = row["booking_id"]
            pool_driver_id = row["captain_id"]
            time_diff= pool_time - entry_time

            #adding time difference to entry
            entry_with_time_diff = entry.copy()  
            entry_with_time_diff["time_diff"] = time_diff  

            #writing good trips to clean file (same trip and driver id and >5 sec time diff)
            if entry_trip_id == pool_trip_id and entry_driver_id == pool_driver_id and abs(time_diff) <= 5:
                json.dump(entry_with_time_diff, clean_file)
                clean_file.write("\n")
                break
                       

print("Matching process completed. Check 'clean_trips.json' for results.")

Matching process completed. Check 'clean_trips.json' for results.


### Get unclean trips

In [4]:
import json

def find_unmatched_trips(new_file_path, clean_file_path, output_file_path):
    #read data from two JSON files
    new_trips = []
    with open(new_file_path, 'r') as new_file:
        for line in new_file:
            new_trips.append(json.loads(line))

    clean_trips = []
    with open(clean_file_path, 'r') as clean_file:
        for line in clean_file:
            clean_trips.append(json.loads(line))

    #extract trip IDs from clean_trips
    clean_trip_ids = {trip['trip_id'] for trip in clean_trips}

    #extract captain engagement times from filtered_pooling_data
    captain_engagement_times = {row['booking_id']: float(row['captain_engagement_time'] * 60) for _, row in filtered_pooling_data.iterrows()}


    unmatched_trips = []
    for trip in new_trips:
        if trip['trip_id'] not in clean_trip_ids:
            trip_id = trip['trip_id']
            if trip_id in captain_engagement_times:
                
                engagement_time = captain_engagement_times[trip_id]
                traj_time = trip['time']
                time_diff= engagement_time - traj_time

                trip['time_diff'] = time_diff
            
            unmatched_trips.append(trip)


    with open(output_file_path, 'w') as output_file:
        for trip in unmatched_trips:
            json.dump(trip, output_file)
            output_file.write('\n')


traj_file_path = 'fixed_traj_new.json'
clean_file_path = 'clean_trips.json'
unclean_file_path = 'unclean_trips.json'


find_unmatched_trips(traj_file_path, clean_file_path, unclean_file_path)